# Advanced prompts

In this notebook, we will explore some advanced prompt patterns:  
+ Few-shots prompt
+ Chain of Thought prompt
+ Program-Aided Language Models (PAL)
+ REACT
+ CO-STAR

## Setup
#### Load the API key and relevant Python libraries.

In [0]:
!pip install -qU openai
dbutils.library.restartPython()

In [0]:
import os
import openai
from openai import OpenAI
from openai import AzureOpenAI

openai.api_type = "azure"
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_version = "2023-07-01-preview"
openai.api_base = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
#openai.api_base = "https://chatgpt-summarization.openai.azure.com/"

#llm_model_name = "gpt-35-turbo"
#llm_deploy_name = "model-gpt-35-turbo"

llm_model_name = "gpt-4o"
llm_deploy_name = "model-gpt-4o"

client = AzureOpenAI(api_key=openai.api_key,
                     api_version=openai.api_version,
                     azure_endpoint=openai.api_base,
                     )

def ask(prompt):
    try:
        chat_completion = client.chat.completions.create(
            model=llm_deploy_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ],
            )
        return(chat_completion.choices[0].message.content)
    except openai.error.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
    except openai.error.AuthenticationError as e:
        print(f"OpenAI API returned an Authentication Error: {e}")
    except openai.error.APIConnectionError as e:
        print(f"Failed to connect to OpenAI API: {e}")
    except openai.error.InvalidRequestError as e:
        print(f"Invalid Request Error: {e}")
    except openai.error.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
    except openai.error.ServiceUnavailableError as e:
        print(f"Service Unavailable: {e}")
    except openai.error.Timeout as e:
        print(f"Request timed out: {e}")
    except:
        print("An exception has occured.")

### Few-shots prompt

Few-shots prompting means that along with the instruction, you input demonstrations of how the task can be solved. 

##### Zero-shot prompting (without few-shot prompting):

In [0]:
ask("Please analyze the sentiment of the following sentence: 'This phone lasts all day on a single charge. It’s amazing!'")

##### With few-shot prompting:

In [0]:
query = """
I will provide you with a series of sentences. 
After each sentence, please analyze the sentiment expressed and categorize it as “Positive,” “Negative,” or “Neutral.”

Q) "I absolutely love the friendly service at that restaurant." 
A) {Sentiment: "Positive"}
Q) "The movie was a complete waste of time and money." 
A) {Sentiment: "Negative"}
Q) "It’s a typical day, nothing much happening." 
A) {Sentiment: "Neutral"}

Now, please analyze the sentiment of the following sentence:

"This phone lasts all day on a single charge. It’s amazing!"
"""

ask(query)

### Chain of Thought prompt

The main idea of Chain-of-Thought (CoT) prompting is that LLMs handle complex tasks better if they are divided into intermediate steps.

##### Without Chain of Thought:

In [0]:
query = """
Question: What will be the atomic number of the next element to be added to the periodic table of elements?
"""

ask(query)

##### With Chain of Thought:

In [0]:
query = """
- Every element we add to the periodic table of elements has a bigger atomic number comparing to the successor.
- The next element's atomic number adds one to the atomic number of previous one in the periodic table.
- Oganesson, the last element to be added, has the atomic number of 118.

Question: What will be the atomic number of the next element to be added to the periodic table of elements?
"""

ask(query)

### [Program-Aided Language Models (PAL)](https://reasonwithpal.com/)

PAL is a method, where language models generate programs from natural language descriptions, which are then executed by a programmatic runtime like a Python interpreter. That leads to a more accurate result.

##### Without PAL:

In [0]:
ask("There were nine computers in the server room. Five more computers were installed each day, since Monday. How many computers are now in the server room?")

##### With PAL:

In [0]:
query = """
There were nine computers in the server room last Monday. 
Five more computers were installed each day, until today. 
How many computers are now in the server room?
create a Python code to answer the question, execute it, and print the result
Explain each step.
"""

print(ask(query))

### [REACT](https://arxiv.org/pdf/2210.03629)

ReAct is a prompting technique that combines reasoning, that  helps the model induce, track, and update action plans as well as handle
exceptions, and actions, that allow it to interface with and gather additional information
from external sources.

##### Without REACT:

In [0]:
query = """
Emma took a 80 minutes plane ride to Seattle. After spending a quarter of hour in a cab, She then took a two hours train ride to Portland, 
but the train was 5 minutes late. Initially, she would only wait for 15 minutes in the train station before the train departs. 
She then took an Uber for the airport, which took 1/2 of hour. Then, after 1 hour waiting in the airport, she took a plane to Vancouver 
that lasted half of the time she spent on the first transportation mode. 
How long did it take her to get to Vancouver?
"""

print(ask(query))

##### With REACT:

In [0]:
query = """
Emma took a 80 minutes plane ride to Seattle. After spending a quarter of hour in a cab, She then took a two hours train ride to Portland, 
but the train was 5 minutes late. Initially, she would only wait for 15 minutes in the train station before the train departs. 
She then took an Uber for the airport, which took 1/2 of hour. Then, after 1 hour waiting in the airport, she took a plane to Vancouver 
that lasted half of the time she spent on the first transportation mode. 
How long did it take her to get to Vancouver? 
Use the REACT framework to show your reasoning.
"""

print(ask(query))

### CO-STAR  

Effective prompt structuring is crucial for eliciting optimal responses from an LLM. The CO-STAR framework is a handy template for structuring prompts.  

It considers all the key aspects that influence the effectiveness and relevance of an LLM’s response, leading to more optimal responses.  

##### Without CO-STAR:

In [0]:
ask("Write a LinkedIn post to advertise Raiffeisen new product called voiceBot, to allow electronic banking transactions via voice.")

##### With CO-STAR:

In [0]:
query = """
# CONTEXT #
I want to advertise my company’s new product. My company’s name is Raiffeisen and the product is called VoiceBot, that allow electronic banking transactions via voice.

# OBJECTIVE #
Create a LinkedIn post for me, which aims to get people to click on the link to read it.

# STYLE #
Follow the writing style of successful companies on the same sector that advertise similar products.

# TONE #
Persuasive

# AUDIENCE #
My company’s audience profile on LinkedIn is typically the older generation. Tailor your post to target what this audience typically looks out for in hair products.

# RESPONSE #
The LinkedIn post
"""

ask(query)